<a href="https://colab.research.google.com/github/narayananv10/DeepLearning/blob/master/J054_Lab10C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fine-tuning the top layers of a a pre-trained network
To further improve our previous result, we can try to "fine-tune" the last convolutional block of the VGG16 model alongside the top-level classifier. Fine-tuning consist in starting from a trained network, then re-training it on a new dataset using very small weight updates. In our case, this can be done in 3 steps:

1. instantiate the convolutional base of VGG16 and load its weights
2. add our previously defined fully-connected model on top, and load its weights
3. freeze the layers of the VGG16 model up to the last convolutional block

![alt text](http://upscfever.com/upsc-fever/images/fine-tuning.png)

In [1]:
#code part 1
from bs4 import BeautifulSoup
import numpy as np
import requests
import cv2
import PIL.Image
import urllib

page = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n04194289")#ship synset
print(page.content)

# BeautifulSoup is an HTML parsing library

soup = BeautifulSoup(page.content, 'html.parser')#puts the content of the website into the soup variable, each url on a different line
#print(soup)
#print(soup.prettify())

#code part 1.1
bikes_page = requests.get("http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n02834778")#bicycle synset
print(bikes_page.content)

# BeautifulSoup is an HTML parsing library
from bs4 import BeautifulSoup
bikes_soup = BeautifulSoup(bikes_page.content, 'html.parser')#puts the content of the website into the soup variable, each url on a different line

#code part 2
str_soup=str(soup)#convert soup to string so it can be split
type(str_soup)
split_urls=str_soup.split('\r\n')#split so each url is a different possition on a list
print(len(split_urls))#print the length of the list so you know how many urls you have

#code part 2.2
bikes_str_soup=str(bikes_soup)#convert soup to string so it can be split
type(bikes_str_soup)
bikes_split_urls=bikes_str_soup.split('\r\n')#split so each url is a different possition on a list
print(len(bikes_split_urls))

#code part 3
#check if all the images where stored on the files system
!mkdir /content/train #create the Train folder
!mkdir /content/train/ships #create the ships folder
!mkdir /content/train/bikes #create the bikes folder
!mkdir /content/validation
!mkdir /content/validation/ships #create the ships folder
!mkdir /content/validation/bikes #create the bikes folder
#!ls /content/train/ships #list the files inside ships
!mkdir /content/test/ 
!mkdir /content/test/test #list the files inside test

#code part 4
img_rows, img_cols = 32, 32 #number of rows and columns to convert the images to
input_shape = (img_rows, img_cols, 3)#format to store the images (rows, columns,channels) called channels last

def url_to_image(url):
	# download the image, convert it to a NumPy array, and then read
	# it into OpenCV format
	resp = urllib.request.urlopen(url)
	image = np.asarray(bytearray(resp.read()), dtype="uint8")
	image = cv2.imdecode(image, cv2.IMREAD_COLOR)
 
	# return the image
	return image

n_of_training_images=150#the number of training images to use
for progress in range(n_of_training_images):#store all the images on a directory
    # Print out progress whenever progress is a multiple of 20 so we can follow the
    # (relatively slow) progress
    if(progress%20==0):
        print(progress)
    if not split_urls[progress] == None:
      try:
        I = url_to_image(split_urls[progress])
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/train/ships/img'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)

      except:
        None

#do the same for bikes:
for progress in range(n_of_training_images):#store all the images on a directory
    # Print out progress whenever progress is a multiple of 20 so we can follow the
    # (relatively slow) progress
    if(progress%20==0):
        print(progress)
    if not bikes_split_urls[progress] == None:
      try:
        I = url_to_image(bikes_split_urls[progress])
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/train/bikes/img'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)

      except:
        None
        
        
#Validation data:

for progress in range(50):#store all the images on a directory
    # Print out progress whenever progress is a multiple of 20 so we can follow the
    # (relatively slow) progress
    if(progress%20==0):
        print(progress)
    if not split_urls[progress] == None:
      try:
        I = url_to_image(split_urls[n_of_training_images+progress])#get images that are different from the ones used for training
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/validation/ships/img'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)

      except:
        None

#do the same for bikes:
for progress in range(50):#store all the images on a directory
    # Print out progress whenever progress is a multiple of 20 so we can follow the
    # (relatively slow) progress
    if(progress%20==0):
        print(progress)
    if not bikes_split_urls[progress] == None:
      try:
        I = url_to_image(bikes_split_urls[n_of_training_images+progress])#get images that are different from the ones used for training
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/validation/bikes/img'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)

      except:
        None
        
print("\nTRAIN:\n")          
print("\nlist the files inside ships directory:\n")        
!ls /content/train/ships #list the files inside ships
print("\nlist the files inside bikes directory:\n")
!ls /content/train/bikes #list the files inside bikes
print("\nVALIDATION:\n")
print("\nlist the files inside ships directory:\n")        
!ls /content/validation/ships #list the files inside ships
print("\nlist the files inside bikes directory:\n")
!ls /content/validation/bikes #list the files inside bikes   

for progress in range(50):#store all the images on a directory
    # Print out progress whenever progress is a multiple of 20 so we can follow the
    # (relatively slow) progress
    if(progress%20==0):
        print(progress)
    if not split_urls[progress] == None:
      try:
        I = url_to_image(split_urls[n_of_training_images+progress+50])#get images that are different from the ones used for training
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/test/test/ships_'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)

      except:
        None

#do the same for bikes:
for progress in range(50):#store all the images on a directory
    # Print out progress whenever progress is a multiple of 20 so we can follow the
    # (relatively slow) progress
    if(progress%20==0):
        print(progress)
    if not bikes_split_urls[progress] == None:
      try:
        I = url_to_image(bikes_split_urls[n_of_training_images+progress+50])#get images that are different from the ones used for training
        if (len(I.shape))==3: #check if the image has width, length and channels
          save_path = '/content/test/test/bikes_'+str(progress)+'.jpg'#create a name of each image
          cv2.imwrite(save_path,I)

      except:
        None



b"http://static.flickr.com/85/233578996_8f74463083.jpg\r\nhttp://farm3.static.flickr.com/2317/2472759369_f4cc1ff8a0.jpg\r\nhttp://static.flickr.com/2173/2071797511_200e3e4d2e.jpg\r\nhttp://farm2.static.flickr.com/1019/534041866_3aa182311c.jpg\r\nhttp://static.flickr.com/88/231940102_7beb5ecf3e.jpg\r\nhttp://farm4.static.flickr.com/3138/2724596903_d74dd64821.jpg\r\nhttp://static.flickr.com/174/453920264_9406e9972c.jpg\r\nhttp://farm2.static.flickr.com/1419/1408865234_caf87d02e3.jpg\r\nhttp://farm1.static.flickr.com/147/423542865_c091691a6b.jpg\r\nhttp://farm3.static.flickr.com/2074/2367492616_272bf6af6a.jpg\r\nhttp://farm2.static.flickr.com/1081/1365111200_8d105d9fb1.jpg\r\nhttp://farm3.static.flickr.com/2269/2348294071_94fc609f4a.jpg\r\nhttp://static.flickr.com/1016/1443997081_b5eb875055.jpg\r\nhttp://www.walleniuslines.com/img_upload/maersk_teal.jpg\r\nhttp://www.fondear.org/infonautic/Barco/Barco_Navegando/Contenedores_Peligro/sea_container_ship2.jpg\r\nhttp://static.flickr.com/198/4

In [0]:
import os
import pandas as pd
# Path to the train folder
original_test = '/content/test/test'
 
filenames = os.listdir(original_test)
categories = []
for filename in filenames:
    category = filename.split('_')[0]
    if category == 'ships':
        categories.append('ships')
    else:
        categories.append('bikes')

data_test = pd.DataFrame({'filename':filenames,'label':categories})

data_test.to_csv("original_test.csv",index=False)

#Fine tuning using VGG16

In [3]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 50
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')



Using TensorFlow backend.


Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.


In [4]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.


In [5]:
from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.VGG16(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

58892288/58889256 [==============================] - 2s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=50, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/50
11/11 [==============================] - 11s 1s/step - loss: 0.7714 - accuracy: 0.4912 - val_loss: 0.7943 - val_accuracy: 0.3730
Epoch 2/50
11/11 [==============================] - 5s 477ms/step - loss: 0.7578 - accuracy: 0.5146 - val_loss: 0.7794 - val_accuracy: 0.3717
Epoch 3/50
11/11 [==============================] - 5s 477ms/step - loss: 0.7903 - accuracy: 0.4561 - val_loss: 0.7666 - val_accuracy: 0.3751
Epoch 4/50
11/11 [==============================] - 5s 473ms/step - loss: 0.7252 - accuracy: 0.5511 - val_loss: 0.7745 - val_accuracy: 0.3717
Epoch 5/50
11/11 [==============================] - 5s 471ms/step - loss: 0.7236 - accuracy: 0.5904 - val_loss: 0.7614 - val_accuracy: 0.3730
Epoch 6/50
11/11 [==============================] - 5s 475ms/step - loss: 0.7836 - accuracy: 0.5322 - val_loss: 0.7604 - val_accuracy: 0.3751
Epoch 7/50
11/11 [==============================] - 5s 467ms/step - loss: 0.7375 - accuracy: 0.5556 - val_loss: 0.7377 - val_accuracy: 0.3717
Epoch 8/

#Fine tuning with ResNet50

In [6]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.resnet50.ResNet50(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:150]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/5
11/11 [==============================] - 10s 879ms/step - loss: 1.2497 - accuracy: 0.4971 - val_loss: 0.8033 - val_accuracy: 0.4222
Epoch 2/5
11/11 [==============================] - 5s 493ms/step - loss: 1.0912 - accuracy: 0.5146 - val_loss: 0.6990 - val_accuracy: 0.4281
Epoch 3/5
11/11 [==============================] - 5s 489ms/step - loss: 0.9770 - accuracy: 0.5497 - val_loss: 0.7500 - val_accuracy: 0.4511
Epoch 4/5
11/11 [==============================] - 5s 481ms/step - loss: 1.0015 - accuracy: 0.5439 - val_loss: 0.6867 - val_accuracy: 0.6133
Epoch 5/5
11/11 [==============================] - 5s 489ms/step - loss: 0.9862 - accuracy: 0.4886 - val_loss: 0.6469 - val_accuracy: 0.6773


#Fine tuning with Xception

In [7]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.xception.Xception(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
83689472/83683744 [==============================] - 3s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/5
11/11 [==============================] - 13s 1s/step - loss: 0.7097 - accuracy: 0.4854 - val_loss: 0.6995 - val_accuracy: 0.4062
Epoch 2/5
11/11 [==============================] - 6s 505ms/step - loss: 0.7290 - accuracy: 0.5556 - val_loss: 0.6919 - val_accuracy: 0.3901
Epoch 3/5
11/11 [==============================] - 6s 504ms/step - loss: 0.7122 - accuracy: 0.5556 - val_loss: 0.6990 - val_accuracy: 0.3705
Epoch 4/5
11/11 [==============================] - 5s 491ms/step - loss: 0.6966 - accuracy: 0.5614 - val_loss: 0.7144 - val_accuracy: 0.3728
Epoch 5/5
11/11 [==============================] - 5s 497ms/step - loss: 0.7011 - accuracy: 0.5455 - val_loss: 0.7042 - val_accuracy: 0.4096


#Fine tuning VGG19

In [8]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.vgg19.VGG19(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
80142336/80134624 [==============================] - 3s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/5
11/11 [==============================] - 5s 487ms/step - loss: 0.7557 - accuracy: 0.4971 - val_loss: 0.6576 - val_accuracy: 0.7460
Epoch 2/5
11/11 [==============================] - 5s 474ms/step - loss: 0.7602 - accuracy: 0.4620 - val_loss: 0.6543 - val_accuracy: 0.7434
Epoch 3/5
11/11 [==============================] - 5s 478ms/step - loss: 0.7314 - accuracy: 0.4737 - val_loss: 0.6383 - val_accuracy: 0.7480
Epoch 4/5
11/11 [==============================] - 5s 464ms/step - loss: 0.6827 - accuracy: 0.5789 - val_loss: 0.6550 - val_accuracy: 0.7457
Epoch 5/5
11/11 [==============================] - 5s 462ms/step - loss: 0.7456 - accuracy: 0.5000 - val_loss: 0.6473 - val_accuracy: 0.7471


#Fine tuning pretrained ResNet101

In [9]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.resnet.ResNet101(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
171450368/171446536 [==============================] - 5s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/5
11/11 [==============================] - 31s 3s/step - loss: 1.1292 - accuracy: 0.4678 - val_loss: 0.6198 - val_accuracy: 0.5755
Epoch 2/5
11/11 [==============================] - 6s 577ms/step - loss: 1.0876 - accuracy: 0.5205 - val_loss: 0.6814 - val_accuracy: 0.5765
Epoch 3/5
11/11 [==============================] - 6s 570ms/step - loss: 0.9273 - accuracy: 0.5497 - val_loss: 0.6560 - val_accuracy: 0.5777
Epoch 4/5
11/11 [==============================] - 6s 564ms/step - loss: 0.8561 - accuracy: 0.5789 - val_loss: 1.0650 - val_accuracy: 0.5754
Epoch 5/5
11/11 [==============================] - 6s 570ms/step - loss: 0.8562 - accuracy: 0.5965 - val_loss: 0.6224 - val_accuracy: 0.5767


#Load pretrained ResNet152

In [10]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.resnet.ResNet152(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
234700800/234698864 [==============================] - 7s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/5
11/11 [==============================] - 50s 5s/step - loss: 1.0395 - accuracy: 0.5146 - val_loss: 0.8266 - val_accuracy: 0.5767
Epoch 2/5
11/11 [==============================] - 8s 703ms/step - loss: 0.8505 - accuracy: 0.5146 - val_loss: 0.9132 - val_accuracy: 0.5731
Epoch 3/5
11/11 [==============================] - 8s 717ms/step - loss: 0.9182 - accuracy: 0.5497 - val_loss: 0.8299 - val_accuracy: 0.4522
Epoch 4/5
11/11 [==============================] - 8s 703ms/step - loss: 0.9119 - accuracy: 0.5341 - val_loss: 0.6978 - val_accuracy: 0.4062
Epoch 5/5
11/11 [==============================] - 8s 720ms/step - loss: 0.7729 - accuracy: 0.6566 - val_loss: 0.7027 - val_accuracy: 0.4382


#Fine tuning ResNet50v2

In [11]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.resnet_v2.ResNet50V2(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
94674944/94668760 [==============================] - 3s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/5
11/11 [==============================] - 15s 1s/step - loss: 0.8509 - accuracy: 0.5057 - val_loss: 0.6649 - val_accuracy: 0.6121
Epoch 2/5
11/11 [==============================] - 6s 523ms/step - loss: 0.9063 - accuracy: 0.4940 - val_loss: 0.7107 - val_accuracy: 0.6157
Epoch 3/5
11/11 [==============================] - 6s 514ms/step - loss: 0.8848 - accuracy: 0.4620 - val_loss: 0.5903 - val_accuracy: 0.6479
Epoch 4/5
11/11 [==============================] - 6s 501ms/step - loss: 0.8615 - accuracy: 0.5146 - val_loss: 0.5997 - val_accuracy: 0.6617
Epoch 5/5
11/11 [==============================] - 6s 508ms/step - loss: 0.7673 - accuracy: 0.5739 - val_loss: 0.6823 - val_accuracy: 0.5606


#Fine tuning with ResNet101v2

In [12]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.resnet_v2.ResNet101V2(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
171319296/171317808 [==============================] - 5s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/5
11/11 [==============================] - 28s 3s/step - loss: 0.9610 - accuracy: 0.4737 - val_loss: 0.7446 - val_accuracy: 0.5606
Epoch 2/5
11/11 [==============================] - 6s 572ms/step - loss: 0.9509 - accuracy: 0.5146 - val_loss: 0.7002 - val_accuracy: 0.4741
Epoch 3/5
11/11 [==============================] - 6s 565ms/step - loss: 0.8935 - accuracy: 0.4795 - val_loss: 0.6768 - val_accuracy: 0.3901
Epoch 4/5
11/11 [==============================] - 6s 569ms/step - loss: 0.8570 - accuracy: 0.5789 - val_loss: 0.7128 - val_accuracy: 0.4246
Epoch 5/5
11/11 [==============================] - 6s 559ms/step - loss: 0.7829 - accuracy: 0.5284 - val_loss: 0.7346 - val_accuracy: 0.4073


#Fine tune ResNetv152

In [13]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.resnet_v2.ResNet152V2(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
234553344/234545216 [==============================] - 7s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/5
11/11 [==============================] - 47s 4s/step - loss: 0.9244 - accuracy: 0.4327 - val_loss: 0.6297 - val_accuracy: 0.5950
Epoch 2/5
11/11 [==============================] - 8s 697ms/step - loss: 0.7573 - accuracy: 0.5455 - val_loss: 0.6497 - val_accuracy: 0.5086
Epoch 3/5
11/11 [==============================] - 8s 703ms/step - loss: 0.8706 - accuracy: 0.4795 - val_loss: 0.6370 - val_accuracy: 0.5616
Epoch 4/5
11/11 [==============================] - 8s 695ms/step - loss: 0.7299 - accuracy: 0.5614 - val_loss: 0.6378 - val_accuracy: 0.4580
Epoch 5/5
11/11 [==============================] - 8s 691ms/step - loss: 0.6785 - accuracy: 0.5602 - val_loss: 0.6734 - val_accuracy: 0.4416


#Fine tuning with Inceptionv3

In [0]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.inception_v3.InceptionV3(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

#Fine tuning with InceptionResNetV2

In [0]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.inception_resnet_v2.InceptionResNetV2(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

#Fine tuning with MobileNet

In [16]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.mobilenet.MobileNet(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


17227776/17225924 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/5
11/11 [==============================] - 9s 838ms/step - loss: 1.3790 - accuracy: 0.5029 - val_loss: 0.6894 - val_accuracy: 0.5435
Epoch 2/5
11/11 [==============================] - 5s 488ms/step - loss: 1.2721 - accuracy: 0.5439 - val_loss: 0.6883 - val_accuracy: 0.5270
Epoch 3/5
11/11 [==============================] - 5s 487ms/step - loss: 1.1735 - accuracy: 0.5497 - val_loss: 0.6964 - val_accuracy: 0.5386
Epoch 4/5
11/11 [==============================] - 5s 481ms/step - loss: 1.0442 - accuracy: 0.5682 - val_loss: 0.7078 - val_accuracy: 0.5949
Epoch 5/5
11/11 [==============================] - 5s 472ms/step - loss: 1.0875 - accuracy: 0.5723 - val_loss: 0.6841 - val_accuracy: 0.4211


#Fine tuning with DenseNet121

In [17]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.densenet.DenseNet121(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
29089792/29084464 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/5
11/11 [==============================] - 37s 3s/step - loss: 1.0073 - accuracy: 0.4620 - val_loss: 1.0263 - val_accuracy: 0.3856
Epoch 2/5
11/11 [==============================] - 6s 576ms/step - loss: 0.9172 - accuracy: 0.4795 - val_loss: 0.8498 - val_accuracy: 0.4614
Epoch 3/5
11/11 [==============================] - 6s 587ms/step - loss: 0.7732 - accuracy: 0.5625 - val_loss: 0.6734 - val_accuracy: 0.4580
Epoch 4/5
11/11 [==============================] - 6s 577ms/step - loss: 0.8038 - accuracy: 0.5723 - val_loss: 0.7757 - val_accuracy: 0.4902
Epoch 5/5
11/11 [==============================] - 6s 583ms/step - loss: 0.7646 - accuracy: 0.5966 - val_loss: 0.7713 - val_accuracy: 0.5263


#Fine tuning with DenseNet169

In [18]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.densenet.DenseNet169(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
51879936/51877672 [==============================] - 2s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/5
11/11 [==============================] - 55s 5s/step - loss: 0.7560 - accuracy: 0.5614 - val_loss: 0.7878 - val_accuracy: 0.4233
Epoch 2/5
11/11 [==============================] - 8s 697ms/step - loss: 0.7402 - accuracy: 0.5380 - val_loss: 0.7887 - val_accuracy: 0.4396
Epoch 3/5
11/11 [==============================] - 8s 687ms/step - loss: 0.7108 - accuracy: 0.5673 - val_loss: 0.8507 - val_accuracy: 0.4568
Epoch 4/5
11/11 [==============================] - 8s 705ms/step - loss: 0.6061 - accuracy: 0.6608 - val_loss: 0.8785 - val_accuracy: 0.4591
Epoch 5/5
11/11 [==============================] - 8s 707ms/step - loss: 0.6338 - accuracy: 0.6818 - val_loss: 0.8597 - val_accuracy: 0.4565


#Fine tuning DenseNet201

In [19]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.densenet.DenseNet201(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
74842112/74836368 [==============================] - 3s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/5
11/11 [==============================] - 68s 6s/step - loss: 0.7978 - accuracy: 0.4795 - val_loss: 0.7204 - val_accuracy: 0.4737
Epoch 2/5
11/11 [==============================] - 8s 740ms/step - loss: 0.7606 - accuracy: 0.5439 - val_loss: 0.7762 - val_accuracy: 0.4879
Epoch 3/5
11/11 [==============================] - 8s 756ms/step - loss: 0.7217 - accuracy: 0.5789 - val_loss: 0.7031 - val_accuracy: 0.5293
Epoch 4/5
11/11 [==============================] - 8s 736ms/step - loss: 0.6543 - accuracy: 0.6140 - val_loss: 0.6452 - val_accuracy: 0.6617
Epoch 5/5
11/11 [==============================] - 9s 779ms/step - loss: 0.6089 - accuracy: 0.6705 - val_loss: 0.6205 - val_accuracy: 0.6957


#Fine tune NasNetLarge

In [20]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.nasnet.NASNetLarge(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
343613440/343610240 [==============================] - 10s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/5
11/11 [==============================] - 110s 10s/step - loss: 0.8461 - accuracy: 0.5170 - val_loss: 0.6996 - val_accuracy: 0.5412
Epoch 2/5
11/11 [==============================] - 10s 899ms/step - loss: 0.7375 - accuracy: 0.5723 - val_loss: 0.7057 - val_accuracy: 0.5086
Epoch 3/5
11/11 [==============================] - 10s 876ms/step - loss: 0.8200 - accuracy: 0.4943 - val_loss: 0.7142 - val_accuracy: 0.5926
Epoch 4/5
11/11 [==============================] - 10s 873ms/step - loss: 0.8203 - accuracy: 0.4880 - val_loss: 0.6901 - val_accuracy: 0.5788
Epoch 5/5
11/11 [==============================] - 10s 875ms/step - loss: 0.7839 - accuracy: 0.5114 - val_loss: 0.7012 - val_accuracy: 0.5412


#Fine tune NasNetMobile

In [21]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.nasnet.NASNetMobile(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
19996672/19993432 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/5
11/11 [==============================] - 71s 6s/step - loss: 1.6952 - accuracy: 0.5322 - val_loss: 2.1728 - val_accuracy: 0.4222
Epoch 2/5
11/11 [==============================] - 9s 774ms/step - loss: 1.4433 - accuracy: 0.5029 - val_loss: 1.6433 - val_accuracy: 0.4062
Epoch 3/5
11/11 [==============================] - 8s 753ms/step - loss: 1.1984 - accuracy: 0.5556 - val_loss: 1.4104 - val_accuracy: 0.4235
Epoch 4/5
11/11 [==============================] - 8s 759ms/step - loss: 1.3422 - accuracy: 0.4386 - val_loss: 1.1836 - val_accuracy: 0.4741
Epoch 5/5
11/11 [==============================] - 8s 752ms/step - loss: 1.0892 - accuracy: 0.5322 - val_loss: 1.2120 - val_accuracy: 0.5229


#Fine tune MobileNetV2

In [22]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# dimensions of our images.
img_width, img_height = 32, 32

train_data_dir = '/content/train'
validation_data_dir = '/content/validation'
nb_train_samples = 189
nb_validation_samples = 59
epochs = 5
batch_size = 16



# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

from keras.models import Model
from keras.layers import Input
# build the VGG16 network

input_tensor = Input(shape=(32,32,3))
base_model = applications.mobilenet_v2.MobileNetV2(weights='imagenet',include_top= False,input_tensor=input_tensor)
top_model = Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))
model = Model(input= base_model.input, output= top_model(base_model.output))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning

# add the model on top of the convolutional base




# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.
Found 187 images belonging to 2 classes.
Found 59 images belonging to 2 classes.


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9412608/9406464 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("se...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., epochs=5, validation_data=<keras.pre..., steps_per_epoch=11, validation_steps=59)`


Epoch 1/5
11/11 [==============================] - 14s 1s/step - loss: 1.0535 - accuracy: 0.5029 - val_loss: 0.6855 - val_accuracy: 0.5584
Epoch 2/5
11/11 [==============================] - 6s 514ms/step - loss: 0.9819 - accuracy: 0.4854 - val_loss: 0.6880 - val_accuracy: 0.5259
Epoch 3/5
11/11 [==============================] - 6s 503ms/step - loss: 0.8783 - accuracy: 0.5795 - val_loss: 0.6903 - val_accuracy: 0.4568
Epoch 4/5
11/11 [==============================] - 6s 502ms/step - loss: 0.8197 - accuracy: 0.6023 - val_loss: 0.6960 - val_accuracy: 0.4741
Epoch 5/5
11/11 [==============================] - 6s 506ms/step - loss: 0.9408 - accuracy: 0.5361 - val_loss: 0.7049 - val_accuracy: 0.5057


#Thanks for completing this notebook